# SPLAT Tutorials: Database Query Tools

## Authors
Adam Burgasser

## Version date
27 May 2023

## Learning Goals
* Explore some of the data spreadsheet manipulation tools built into SPLAT (``splat.database.prepDB``)
* Learn how to use the astroquery wrapper for Vizier to get individual source information (``splat.database.getPhotometry``, ``splat.database.querySimbad``)
* Learn how to use the astroquery wrapper for Simbad to get individual source information (``splat.database.querySimbad``)
* Learn how to use the astroquery wrapper for XMatch to get information for many sources (``splat.database.queryXMatch``)

## Keywords
astroquery, databases

## Companion Content
None

## Summary
In this tutorial, we are going to see how to use the splat.database functions to manage source spreadsheets and query online databases for source informaiton.


In [ ]:
# main splat import
import splat
import splat.database as spdb

# other useful imports
import astropy.units as u
import copy
import numpy as np
import pandas
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# make sure this is at least 2023.05
splat.VERSION

# Prepping datasets

SPLAT useds pandas as its default spreadsheet format. There is a simple tool called prepDB available to manage sets of data to assure one has sufficient informaiton to query online catalogs. We're going to explore a couple of cases based on the targets observed by Terrien et al. (2015), for which there are two .csv files in the SPLAT tutorial directory

In [ ]:
# let's start with a folder of RA & DEC
db = pandas.read_csv(splat.SPLAT_PATH+splat.TUTORIAL_FOLDER+'terrien2015_radec.csv')
db

In [ ]:
# add in the necessary information for queries with prepDB
# this adds in columns for designation and SkyCoord coordinates
db = spdb.prepDB(db)
db

In [ ]:
# alternately let's assume we have a file that contains only designations
db = pandas.read_csv(splat.SPLAT_PATH+splat.TUTORIAL_FOLDER+'terrien2015_designations.csv')
db

In [ ]:
# prepDB will adds in the columns for RA, Dec and SkyCoord coordinates
db = spdb.prepDB(db)
db

# Getting photometry with getPhotometry

The ``splat.database.getPhotometry()`` is a wrapper for astroquery.Vizier, allowing you to query the Vizier network of catalogs to find relevant photometry and other information. This code is particularly well suited for searching on source at a time; for a large number of sources it is probably better to use ``splat.database.queryXMatch()``. To start, let's find 2MASS, SDSS and WISE data for one of the sources in our catalog

In [ ]:
# look at the docstring for this function
spdb.getPhotometry?

In [ ]:
# look to see what input catalogs are available
# the links point to the information page on this catalog in Vizier
spdb.getPhotometry(_,info=True)

In [ ]:
# take the first coordinate and search for 2MASS photometry
# this will search within 30" of the target and 
# return a pandas spreadsheet in order of separation from the original coordinate
srch = spdb.getPhotometry(db['COORDINATES'].iloc[0],catalog='2MASS',radius=30*u.arcsec)
srch

In [ ]:
# if we want to return only the nearest target, set nearest=True
srch = spdb.getPhotometry(db['COORDINATES'].iloc[0],catalog='2MASS',radius=30*u.arcsec,nearest=True)
srch

In [ ]:
# note that the duplicate function spdb.queryVizier does the same thing
srch = spdb.queryVizier(db['COORDINATES'].iloc[0],catalog='2MASS',radius=30*u.arcsec,nearest=True)
srch

In [ ]:
# let's add the 2MASS J, H, and K magnitudes and their uncertainties 
# to our original spreadsheet
# first set up columns
cstring = ['Jmag','e_Jmag','Hmag','e_Hmag','Kmag','e_Kmag','sep']
for c in cstring: 
    db[c] = np.zeros(len(db))

# now add our measurement for the nearest source
for c in cstring: 
    db[c].iloc[0] = srch[c].iloc[0]

db.iloc[0]

In [ ]:
# now let's repeat for the first 30 positions - this will take several seconds!
for i,crd in enumerate(db['COORDINATES'].iloc[:30]):
    srch = spdb.getPhotometry(crd,catalog='2MASS',radius=30.*u.arcsec,nearest=True)
# catch for case where nothing is returned
    if len(srch)>0: 
        for c in cstring: 
            db[c].iloc[i] = srch[c].iloc[0]
db.iloc[:30]

In [ ]:
# you can search on other catalogs in Vizier as well by entering in the catalog reference
# here's an example for TESS Input Catalog (TIC) of Stassun et al. 2019
# see https://cdsarc.unistra.fr/viz-bin/cat/IV/38
cat = 'IV/38/tic'
srch = spdb.getPhotometry(db['COORDINATES'].iloc[0],catalog=cat,radius=30*u.arcsec,nearest=True)
srch

## Exercise

Explore some of the other catalogs that are available in the getPhotometry code

In [ ]:
# enter your code here!



# Querying SIMBAD

The ``splat.database.querySimbad()`` function searches specifically on the Simbad database, and looks a lot like ``splat.database.getPhotometry()``

In [ ]:
# search for SIMBAD sources around the first coordinate in our table
srch = spdb.querySimbad(db['COORDINATES'].iloc[0],radius=5*u.arcminute)
srch

In [ ]:
# see what information we get from this
srch.keys()

In [ ]:
# not all of these sources are what we want, so we can reject by object type
srch = spdb.querySimbad(db['COORDINATES'].iloc[0],radius=5*u.arcminute,reject_type='Galaxy')
srch

In [ ]:
# we can also return the nearest soruce to our coordinate
srch = spdb.querySimbad(db['COORDINATES'].iloc[0],radius=5*u.arcminute,nearest=True)
srch

In [ ]:
# we can also search by the name of the source
srch = spdb.querySimbad('G 158-27',isname=True)
srch

In [ ]:
# there is also a slightly reformated version of the output that can be returned
srch = spdb.querySimbad(db['COORDINATES'].iloc[0],radius=5*u.arcminute,clean=True)
srch

# Query a large collection of sources: queryXMatch

Each of these methods is fine for individual sources, but can be slow for a large list of objects (like our sample database!). Fortunately, the astroquery xmatch function is well suited to this case, and SPLAT as a wrapper for this called ``splat.database.queryXMatch()``. In this case, you input the entire table, and as long as DESIGNATION or RA and DEC columns are provided, it will return the closest match to each source in the catalog


In [ ]:
# let's reload our catalog
db = pandas.read_csv(splat.SPLAT_PATH+splat.TUTORIAL_FOLDER+'terrien2015_radec.csv')
db = spdb.prepDB(db)
db

In [ ]:
# let's get all the 2MASS data for this catalog - it's pretty fast!
db2 = spdb.queryXMatch(db,radius=30.*u.arcsec,catalog='2MASS')
db2

In [ ]:
# maybe we want only a few select columns; we can use the pre-select versions
db2 = spdb.queryXMatch(db,radius=30.*u.arcsec,catalog='2MASS',use_select_columns=True)
db2

In [ ]:
# we can also specific the columns we want to keep
# as long as they are among the columns returned by the catalog
# note that we've dropped the 2MASS prefix
# one of these columns won't work and will throw up a warning
select_columns = ['Jmag','e_Jmag','Hmag','e_Hmag','Kmag','e_Kmag','JUNK']
db2 = spdb.queryXMatch(db,radius=30.*u.arcsec,catalog='2MASS',select_columns=select_columns,use_select_columns=False)
db2

In [ ]:
# we can also query Simbad
db2 = spdb.queryXMatch(db,radius=30.*u.arcsec,catalog='SIMBAD')
db2

In [ ]:
# we can also query other Vizier catalogs
# let's again look at the  TESS Input Catalog (TIC) of Stassun et al. 2019
# I've changed the name of the catalog prefix to make my columns cleaner
cat = 'IV/38/tic'
db2 = spdb.queryXMatch(db,radius=30.*u.arcsec,catalog=cat,prefix='TIC')
db2

In [ ]:
# the great thing about queryXMatch is that searches to multiple catalogs 
# can be strung together to produce an overall dataset
db2 = copy.deepcopy(db)
for cat in ['SDSS','2MASS','ALLWISE','GAIA-EDR3']:
    db2 = spdb.queryXMatch(db2,catalog=cat,radius=30*u.arcsec)
db2

In [ ]:
# lots of columns!
print(list(db2.columns))

In [ ]:
# now we can generate some absolute magnitudes and colors
magnitudes =  ['SDSS_gmag','SDSS_rmag','SDSS_imag','SDSS_zmag','GAIA-EDR3_phot_g_mean_mag','GAIA-EDR3_phot_bp_mean_mag','GAIA-EDR3_phot_rp_mean_mag','2MASS_Jmag','2MASS_Hmag','2MASS_Kmag','ALLWISE_W1mag','ALLWISE_W2mag','ALLWISE_W3mag','ALLWISE_W4mag']
for i,m in enumerate(magnitudes):
    db2['ABSOLUTE_{}'.format(m)] = db2[m]-5.*np.log10(db2['GAIA-EDR3_parallax']/100)
    for m2 in magnitudes[(i+1):]:
        db2['{}-{}'.format(m,m2)] = db2[m]-db2[m2]
print(list(db2.columns[-100:]))


In [ ]:
# display some color-color and color-magnitude plots
combinations = [
    ['ABSOLUTE_GAIA-EDR3_phot_g_mean_mag','GAIA-EDR3_phot_bp_mean_mag-GAIA-EDR3_phot_rp_mean_mag'],
    ['ABSOLUTE_SDSS_imag','SDSS_imag-SDSS_zmag'],
    ['ABSOLUTE_2MASS_Jmag','2MASS_Jmag-2MASS_Kmag'],
    ['ABSOLUTE_ALLWISE_W2mag','2MASS_Jmag-ALLWISE_W2mag'],
    ['2MASS_Jmag-2MASS_Kmag','SDSS_imag-SDSS_zmag'],
    ['2MASS_Jmag-ALLWISE_W2mag','SDSS_rmag-2MASS_Kmag'],
]
fig, axs = plt.subplots(6,figsize=[5,15])
for i,c in enumerate(combinations):
    axs[i].plot(db2[c[1]],db2[c[0]],'o',alpha=0.5)
    axs[i].set_xlabel(c[1])
    axs[i].set_ylabel(c[0])
    axs[i].set_xlim(np.nanquantile(db2[c[1]],[0.05,0.95]))
    axs[i].set_ylim(np.nanquantile(db2[c[0]],[0.95,0.05]))
fig.tight_layout()
    